In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import datapackage
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import statsmodels.api as sm

## investigating volatility linkages between oil, gas, and regionalised coal futures markets ##

## load data from Github into new frame and remove rows with absent data ##

url = 'https://raw.githubusercontent.com/joe-ascroft/phd/master/data/df-combined.csv'
df = pd.read_csv(url)
df["DATE"] = pd.to_datetime(df["DATE"],dayfirst=True)
for col in df.columns[1:]:
    df[col] = pd.to_numeric(df[col],errors='coerce')
    
df = df.dropna()
print(df)

           DATE    NCF    RCF    WTI    HH  Brent  NBP_UK  GBP-USD       NBP
1    2009-01-02  74.35  81.00  46.17  5.41  46.91   54.74   1.4453  7.911572
4    2009-01-05  77.65  84.40  48.61  5.83  49.62   56.66   1.4708  8.333553
5    2009-01-06  81.25  88.50  48.56  6.10  50.53   59.88   1.4919  8.933497
6    2009-01-07  78.65  86.25  42.75  5.89  45.86   58.03   1.5111  8.768913
7    2009-01-08  79.15  84.00  41.68  5.96  44.67   55.74   1.5202  8.473595
...         ...    ...    ...    ...   ...    ...     ...      ...       ...
4254 2020-08-25  49.75  48.90  43.17  2.54  45.86   24.61   1.3148  3.235723
4255 2020-08-26  49.70  48.95  43.21  2.52  45.64   25.10   1.3193  3.311443
4256 2020-08-27  49.80  48.90  42.88  2.52  45.09   26.68   1.3206  3.523361
4257 2020-08-28  50.60  52.20  42.96  2.46  45.81   28.95   1.3344  3.863088
4260 2020-08-31  51.95  52.90  42.61  2.30  45.28   28.95   1.3379  3.873220

[2856 rows x 9 columns]


In [ ]:
## price return calculations ##

for col in df.columns[1:9]:
    df[col+'_r'] = (df[col].pct_change())
df = df.dropna()
print(df)

In [4]:
## markov regime switching for univariate analysis ##

mod = sm.tsa.MarkovRegression(df["HH"], k_regimes=3, switching_variance = True)
res_mod = mod.fit()
res_mod.summary()

/Users/joe/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'


<class 'statsmodels.iolib.summary.Summary'>
"""
                        Markov Switching Model Results                        
==============================================================================
Dep. Variable:                     HH   No. Observations:                 2856
Model:               MarkovRegression   Log Likelihood               -1277.427
Date:                Thu, 11 Nov 2021   AIC                           2578.854
Time:                        16:32:57   BIC                           2650.341
Sample:                             0   HQIC                          2604.632
                               - 2856                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.0977      0.014    152.337      0.000       2.071       2.125
sigma2         0.0810      0.005     15.347      0.000       0.071       0.091
                             Regime 1 parameters                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.8594      0.007    439.235      0.000       2.847       2.872
sigma2         0.0304      0.002     19.112      0.000       0.027       0.034
                             Regime 2 parameters                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.0558      0.019    218.907      0.000       4.019       4.092
sigma2         0.4422      0.017     25.683      0.000       0.408       0.476
                         Regime transition parameters                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p[0->0]        0.9878      0.001   1196.238      0.000       0.986       0.989
p[1->0]        0.0082      0.003      2.675      0.007       0.002       0.014
p[2->0]     1.638e-16        nan        nan        nan         nan         nan
p[0->1]        0.0122      0.000     47.901      0.000       0.012       0.013
p[1->1]        0.9826      0.004    225.804      0.000       0.974       0.991
p[2->1]        0.0065      0.002      2.957      0.003       0.002       0.011
==============================================================================

Warnings:
[1] Covariance matrix calculated using numerical (complex-step) differentiation.
"""

In [ ]:
res_mod.filtered_marginal_probabilities[1].plot(
    title='Probability of being in the high regime', figsize=(12,2))

In [ ]:
plt.plot(df["WTI"])

In [ ]:
## markov regime switching for univariate returns analysis ##

mod = sm.tsa.MarkovRegression(df["GBP-USD"], k_regimes=3, switching_variance = True)
res_mod = mod.fit()
res_mod.summary()